In [39]:
import os
import sys
import numpy as np
import pandas as pd
from numpy import random as npr

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)



In [40]:
SEED = 2021

USERPROFILE_DATASET_PATH = "../data/restaurant_data/userprofile.csv"
USERCUISINE_DATASET_PATH = "../data/restaurant_data/usercuisine.csv"
CHEFMOZCUISINE_DATASET_PATH = "../data/restaurant_data/chefmozcuisine.csv"
RATINGS_DATASET_PATH = "../data/restaurant_data_reformatted/ratings.csv"

## Load and transform restaurants data

Helper functions 

In [41]:
def prepare_user_profile_df(df):
    df = df.drop(["latitude", "longitude"], axis=1)
    df = df.replace("?", pd.NA)
    df = df.fillna(method="bfill")
    return df


def prepare_user_cuisine_df(df):
    df.drop_duplicates()
    df = df.join(pd.get_dummies(df["Rcuisine"]))
    df = df.drop("Rcuisine", axis=1)
    df = df.groupby("userID").sum()
    return df


def load_and_clean_users_df():
    user_profile_df = pd.read_csv(USERPROFILE_DATASET_PATH)
    user_cuisine_df = pd.read_csv(USERCUISINE_DATASET_PATH)
    user_profile_df = prepare_user_profile_df(user_profile_df)
    user_cuisine_df = prepare_user_cuisine_df(user_cuisine_df)

    users_df = pd.merge(user_profile_df, user_cuisine_df, on="userID")
    return users_df


def load_and_prepare_rest_cuisine_df():
    df = pd.read_csv(CHEFMOZCUISINE_DATASET_PATH)
    df = df.drop_duplicates()
    df = df.join(pd.get_dummies(df["Rcuisine"]))
    df = df.drop("Rcuisine", axis=1)
    df = df.groupby("placeID").sum()
    return df



In [42]:
users_df = load_and_clean_users_df()
users_df

,userID,smoker,drink_level,dress_preference,ambience,transport,marital_status,hijos,birth_year,interest,...,Swiss,Tapas,Tea_House,Tex-Mex,Thai,Tibetan,Tunisian,Turkish,Vegetarian,Vietnamese
0,U1001,false,abstemious,informal,family,on foot,single,independent,1989,variety,...,0,0,0,0,0,0,0,0,0,0
1,U1002,false,abstemious,informal,family,public,single,independent,1990,technology,...,0,0,0,0,0,0,0,0,0,0
2,U1003,false,social drinker,formal,family,public,single,independent,1989,none,...,0,0,0,0,0,0,0,0,0,0
3,U1004,false,abstemious,informal,family,public,single,independent,1940,variety,...,0,0,0,0,0,0,0,0,0,0
4,U1005,false,abstemious,no preference,family,public,single,independent,1992,none,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,U1134,false,casual drinker,no preference,family,public,single,independent,1991,variety,...,0,0,0,0,0,0,0,0,0,0
134,U1135,false,casual drinker,informal,family,on foot,single,kids,1988,variety,...,1,1,1,1,1,1,1,1,1,1
135,U1136,true,social drinker,no preference,friends,car owner,single,independent,1990,retro,...,0,0,0,0,0,0,0,0,0,0
136,U1137,false,social drinker,formal,family,public,single,independent,1989,eco-friendly,...,0,0,0,0,0,0,0,0,0,0


In [43]:
rests_df = load_and_prepare_rest_cuisine_df()
rests_df

,Afghan,African,American,Armenian,Asian,Bagels,Bakery,Bar,Bar_Pub_Brewery,Barbecue,...,Soup,Southern,Southwestern,Spanish,Steaks,Sushi,Thai,Turkish,Vegetarian,Vietnamese
placeID,,,,,,,,,,,,,,,,,,,,,
132001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
132002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
132003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
132004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
132005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135105,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
135106,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
135107,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
ratings_df = pd.read_csv(RATINGS_DATASET_PATH)
ratings_df

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2
...,...,...,...,...,...
1156,U1043,132630,1,1,1
1157,U1011,132715,1,1,0
1158,U1068,132733,1,1,0
1159,U1068,132594,1,1,1


Merge ratings dataset, users dataset, and restaurants dataset into single tablem

In [45]:
merged_df = pd.merge(ratings_df, users_df, on="userID")
merged_df = pd.merge(merged_df, rests_df, on="placeID")
merged_df

,userID,placeID,rating,food_rating,service_rating,smoker,drink_level,dress_preference,ambience,transport,...,Soup_y,Southern_y,Southwestern_y,Spanish_y,Steaks_y,Sushi_y,Thai_y,Turkish_y,Vegetarian_y,Vietnamese_y
0,U1077,135085,2,2,2,false,social drinker,elegant,family,public,...,0,0,0,0,0,0,0,0,0,0
1,U1108,135085,1,2,1,false,abstemious,informal,solitary,public,...,0,0,0,0,0,0,0,0,0,0
2,U1081,135085,1,2,1,false,casual drinker,informal,family,public,...,0,0,0,0,0,0,0,0,0,0
3,U1001,135085,0,1,1,false,abstemious,informal,family,on foot,...,0,0,0,0,0,0,0,0,0,0
4,U1056,135085,2,2,2,false,social drinker,informal,family,on foot,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
868,U1006,132922,2,1,2,true,social drinker,no preference,friends,car owner,...,0,0,0,0,0,0,0,0,0,0
869,U1003,132937,2,2,1,false,social drinker,formal,family,public,...,0,0,0,0,0,0,0,0,0,0
870,U1027,132937,1,1,1,true,social drinker,no preference,family,public,...,0,0,0,0,0,0,0,0,0,0
871,U1029,132937,1,1,1,true,casual drinker,formal,family,public,...,0,0,0,0,0,0,0,0,0,0


## Generate synthetic data

In [46]:
from sdv.tabular import GaussianCopula


def fit_syn_generator(df):
    model = GaussianCopula()
    df = df.drop(["userID", "placeID"], axis=1) # Drop ids
    df = df.astype("int64", errors="ignore") # Convert all numbers to int64
    model.fit(df.copy())
    return model



In [47]:
np.random.seed(SEED)
syn_data_generator = fit_syn_generator(merged_df)

/Users/vldpro/Workspace/university/recsys/.venv/lib/python3.7/site-packages/scipy/stats/_continuous_distns.py:4798: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
/Users/vldpro/Workspace/university/recsys/.venv/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:2407: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/Users/vldpro/Workspace/university/recsys/.venv/lib/python3.7/site-packages/copulas/univariate/truncated_gaussian.py:43: RuntimeWarning: divide by zero encountered in double_scalars
  a = (self.min - loc) / scale
/Users/vldpro/Workspace/university/recsys/.venv/lib/python3.7/site-packages/copulas/univariate/truncated_gaussian.py:44: RuntimeWarning: invalid value encountered in double_scalars
  b = (self.max - loc) / scale
/Users/vldpro/Workspace/university/recsys/.venv/lib/python3.7/site-packages/scipy/stats/_continuous_distns.py:547: RuntimeWarning: invalid value encountered in sqrt
  

In [48]:
np.random.seed(SEED)
syn_merged_df = syn_data_generator.sample(10_000)
syn_merged_df.head()

,rating,food_rating,service_rating,smoker,drink_level,dress_preference,ambience,transport,marital_status,hijos,...,Soup_y,Southern_y,Southwestern_y,Spanish_y,Steaks_y,Sushi_y,Thai_y,Turkish_y,Vegetarian_y,Vietnamese_y
0,1,1,1,false,social drinker,elegant,family,public,single,independent,...,0,0,0,0,0,0,0,0,0,0
1,1,2,2,false,casual drinker,informal,family,public,single,dependent,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,false,social drinker,elegant,family,public,single,independent,...,0,0,0,0,0,0,0,0,0,0
3,1,1,2,false,social drinker,elegant,family,public,single,dependent,...,0,0,0,0,0,0,0,0,0,0
4,1,2,2,false,casual drinker,informal,family,public,single,independent,...,0,0,0,0,0,0,0,0,0,0


## Fit DeepFM model

In [49]:
import torch
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names

from modules.models import DeepFmModel

In [50]:
dense_feat_names = ["height", "weight", "birth_year"]
sparse_feat_names = [
    c for c in list(syn_merged_df.columns) if c not in [
        "rating",
        "food_rating",
        "service_rating",
        "weight",
        "height",
        "birth_year"
    ]
]

In [51]:
class DeepFMDataLoader:
    def __init__(self, *, sparse_features, dense_features):
        self._sparse_feats = sparse_features
        self._dense_feats = dense_features
        
    def load(self, dataset):
        nn_input = pd.DataFrame()
        nn_input[self._sparse_feats] = dataset[self._sparse_feats]
        nn_input[self._dense_feats] = dataset[self._dense_feats]
        
        for feat in self._sparse_feats:
            encoder = LabelEncoder()
            nn_input[feat] = encoder.fit_transform(nn_input[feat])
            
        mms = MinMaxScaler(feature_range=(0,1))
        nn_input[self._dense_feats] = mms.fit_transform(nn_input[self._dense_feats])
        
        # problems may be here
        sparse_feature_columns = [
            SparseFeat(feat, vocabulary_size=nn_input[feat].nunique(), embedding_dim=4) 
            for i, feat in enumerate(self._sparse_feats)
        ]

        dense_feature_columns = [DenseFeat(feat, 1,) for feat in self._dense_feats]
        
        dnn_feat_cols = sparse_feature_columns + dense_feature_columns
        linear_feat_cols = sparse_feature_columns + dense_feature_columns
        
        feat_names = get_feature_names(linear_feat_cols + dnn_feat_cols)
        return nn_input, dnn_feat_cols, linear_feat_cols, feat_names


data_loader = DeepFMDataLoader(sparse_features=sparse_feat_names, dense_features=dense_feat_names)
nn_train_input, dnn_feats, lin_feats, feat_names = data_loader.load(syn_merged_df)

In [52]:
def _cross_join(df1, df2):
    df1["_join_key"] = 0
    df2["_join_key"] = 0
    merged_df = df1.merge(df2, on="_join_key")
    merged_df = merged_df.drop("_join_key", axis=1)
    return merged_df


def nn_prepare_data_for_rating_matrix(users_df, rests_df):
    users_df = users_df.drop("userID", axis=1)
    users_df["user_id"] = range(0, len(users_df))
    rests_df["item_id"] = range(0, len(rests_df))
    user_rest_long_table = _cross_join(users_df, rests_df)
    return user_rest_long_table
    

user_rest_long_table = nn_prepare_data_for_rating_matrix(users_df.copy(), rests_df.copy())

In [53]:
nn_user_rest_long_table, _dnn_feats, _lin_feats, _feat_names = data_loader.load(user_rest_long_table)

In [54]:
def merge_feats(feats_a, feats_b):
    assert len(feats_a) == len(feats_b)
    merged = []
    for feat_a, feat_b in zip(feats_a, feats_b):
        if isinstance(feat_a, DenseFeat):
            continue
        if feat_a.vocabulary_size >= feat_b.vocabulary_size:
            merged.append(feat_a)
        else:
            merged.append(feat_b)
    return merged


_merged_feats = merge_feats(dnn_feats, _dnn_feats)

In [55]:
def train_deepfm(feats, feat_names, x, y):
    deepfm = DeepFmModel(feats, feats, feat_names)
    train_set, test_set = train_test_split(x, test_size=0.2)
    deepfm.train(train_set, target_values=y[:len(train_set)])
    return deepfm
    

In [56]:
np.random.seed(SEED)

deepfm = train_deepfm(_merged_feats, feat_names, x=nn_train_input, y=syn_merged_df["rating"].values)

cpu
Train on 6400 samples, validate on 1600 samples, 25 steps per epoch
Epoch 1/10
9s - loss:  0.8544 - mse:  0.8544 - val_mse:  0.7560
Epoch 2/10
8s - loss:  0.6917 - mse:  0.6917 - val_mse:  0.7563
Epoch 3/10
7s - loss:  0.6866 - mse:  0.6866 - val_mse:  0.7568
Epoch 4/10
7s - loss:  0.6882 - mse:  0.6882 - val_mse:  0.7580
Epoch 5/10
11s - loss:  0.6844 - mse:  0.6844 - val_mse:  0.7605
Epoch 6/10
10s - loss:  0.6843 - mse:  0.6843 - val_mse:  0.7656
Epoch 7/10
8s - loss:  0.6831 - mse:  0.6831 - val_mse:  0.7592
Epoch 8/10
9s - loss:  0.6807 - mse:  0.6807 - val_mse:  0.7629
Epoch 9/10
8s - loss:  0.6819 - mse:  0.6819 - val_mse:  0.7614
Epoch 10/10
7s - loss:  0.6808 - mse:  0.6808 - val_mse:  0.7623


In [57]:
np.random.seed(SEED)

class NNModelWrapper:
    def __init__(self, trained_nn):
        self._nn = trained_nn

    def predict_rating_matrix(self, nn_input, merged_df):
        y = self._nn.predict(nn_input)
        result = pd.DataFrame()
        result["rating"] = y.reshape((len(y),))
        result["user_id"] = merged_df["user_id"]
        result["item_id"] = merged_df["item_id"]
        output_matrix = result.pivot(index="user_id", columns="item_id", values="rating")
        return output_matrix

model_wrapper = NNModelWrapper(deepfm)
deepfm_rating_matrix = model_wrapper.predict_rating_matrix(nn_user_rest_long_table, user_rest_long_table)
deepfm_rating_matrix

item_id,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,768
user_id,,,,,,,,,,,,,,,,,,,,,
0,1.359418,1.409055,1.429519,1.409055,1.400598,1.359418,1.360261,1.429519,1.429519,1.351651,...,1.444863,1.444863,1.437241,1.285995,1.453349,1.283507,1.453349,1.360478,1.394664,1.360211
1,1.476742,1.528429,1.549968,1.528429,1.519747,1.476742,1.477667,1.549968,1.549968,1.468826,...,1.565328,1.565328,1.557575,1.399805,1.574050,1.397049,1.574050,1.477902,1.513446,1.477610
2,1.695808,1.751140,1.774545,1.751140,1.742068,1.695808,1.696861,1.774545,1.774545,1.687616,...,1.789855,1.789855,1.781860,1.612639,1.798987,1.609455,1.798987,1.697131,1.735044,1.696792
3,1.638222,1.691971,1.714499,1.691971,1.683081,1.638222,1.639194,1.714499,1.714499,1.630136,...,1.729718,1.729718,1.721817,1.557782,1.738661,1.554857,1.738661,1.639451,1.676271,1.639129
4,1.342126,1.391729,1.412158,1.391729,1.383280,1.342126,1.342961,1.412158,1.412158,1.334358,...,1.427491,1.427491,1.419868,1.268767,1.435971,1.266302,1.435971,1.343178,1.377324,1.342911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,1.527310,1.579746,1.601584,1.579746,1.571000,1.527310,1.528230,1.601584,1.601584,1.519320,...,1.616881,1.616881,1.609064,1.449122,1.625673,1.446370,1.625673,1.528475,1.564424,1.528169
134,0.705029,0.738766,0.751377,0.738766,0.731960,0.705029,0.705415,0.751377,0.751377,0.698516,...,0.766621,0.766621,0.760094,0.658724,0.773361,0.657810,0.773361,0.705473,0.729579,0.705426
135,1.677848,1.732594,1.755774,1.732594,1.723570,1.677848,1.678908,1.755774,1.755774,1.669714,...,1.771066,1.771066,1.763122,1.595652,1.780143,1.592456,1.780143,1.679169,1.716775,1.678841


In [58]:
deepfm_rating_matrix = np.around(deepfm_rating_matrix)
deepfm_rating_matrix

item_id,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,768
user_id,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,...,2.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0
2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
3,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0
134,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
135,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [59]:
def make_deepfm_rating_matrix(syn_df, users_df, rests_df):
    dense_feat_names = ["height", "weight", "birth_year"]
    sparse_feat_names = [
        c for c in list(syn_df.columns) if c not in [
            "rating",
            "food_rating",
            "service_rating",
            "weight",
            "height",
            "birth_year"
        ]
    ]

    data_loader = DeepFMDataLoader(sparse_features=sparse_feat_names, dense_features=dense_feat_names)
    nn_train_input, dnn_feats, lin_feats, feat_names = data_loader.load(syn_df)
    user_rest_long_table = nn_prepare_data_for_rating_matrix(users_df.copy(), rests_df.copy())
    
    nn_user_rest_long_table, _dnn_feats, _lin_feats, _feat_names = data_loader.load(user_rest_long_table)
    _merged_feats = merge_feats(dnn_feats, _dnn_feats)
    deepfm = train_deepfm(_merged_feats, feat_names, x=nn_train_input, y=syn_df["rating"].values)

    model_wrapper = NNModelWrapper(deepfm)
    deepfm_rating_matrix = model_wrapper.predict_rating_matrix(nn_user_rest_long_table, user_rest_long_table)
    return deepfm_rating_matrix

In [60]:
np.random.seed(SEED)
_mtx = make_deepfm_rating_matrix(syn_merged_df, users_df, rests_df)

cpu
Train on 6400 samples, validate on 1600 samples, 25 steps per epoch
Epoch 1/10
8s - loss:  0.8544 - mse:  0.8544 - val_mse:  0.7560
Epoch 2/10
8s - loss:  0.6917 - mse:  0.6917 - val_mse:  0.7563
Epoch 3/10
8s - loss:  0.6866 - mse:  0.6866 - val_mse:  0.7568
Epoch 4/10
8s - loss:  0.6882 - mse:  0.6882 - val_mse:  0.7580
Epoch 5/10
8s - loss:  0.6844 - mse:  0.6844 - val_mse:  0.7605
Epoch 6/10
7s - loss:  0.6843 - mse:  0.6843 - val_mse:  0.7656
Epoch 7/10
7s - loss:  0.6831 - mse:  0.6831 - val_mse:  0.7592
Epoch 8/10
7s - loss:  0.6807 - mse:  0.6807 - val_mse:  0.7629
Epoch 9/10
8s - loss:  0.6819 - mse:  0.6819 - val_mse:  0.7614
Epoch 10/10
7s - loss:  0.6808 - mse:  0.6808 - val_mse:  0.7623


## Make similarity matrix

Creating a similarity matrix between users and restaurants based on users' food preference and restaurants' cuisines.

In [61]:
from sklearn.metrics.pairwise import cosine_similarity


def make_similarity_matrix(users_df, rests_df):
    users_food_pref_df = users_df[rests_df.columns]
    return cosine_similarity(users_food_pref_df, rests_df)



In [62]:
sim_matrix = make_similarity_matrix(users_df, rests_df)
sim_matrix = sim_matrix * 2 # Rating is in range [0, 2]
sim_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Experiments

In [63]:
import os
import sys
import importlib
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from modules import models, evaluator, trainers, utils
importlib.reload(models)
importlib.reload(evaluator)
importlib.reload(trainers)

<module 'modules.trainers' from '/Users/vldpro/Workspace/university/recsys/modules/trainers.py'>

In [64]:
class ResponseFunction:
    def __init__(self, heu_matrix, nn_matrix, noise_matrix):
        assert heu_matrix.shape == nn_matrix.shape
        self._heu_matrix = heu_matrix
        self._nn_matrix = nn_matrix
        self._noise_matrix = noise_matrix
        
    def __call__(self, a1: float, a2: float):
        a3 = max(0.0, 1 - a1 - a2)
        return (
            a1 * self._heu_matrix
            + a2 * self._nn_matrix
            + a3 * npr.normal(0, 1, size=self._heu_matrix.shape)
        )   

In [ ]:
_resp_fn_config = evaluator.ResponseFunctionConfig(
    factory=ResponseFunction,
    args=[sim_matrix, deepfm_rating_matrix, npr.normal(0, 1, size=sim_matrix.shape)]
)

_evaluators = [
    evaluator.TrainTestExecutorConfig(
        factory=trainers.AutoRecTrainTestExecutor,
        args={"config": {"epoch": 50}},
        model_name="autorec"
    ),
    evaluator.TrainTestExecutorConfig(
        factory=trainers.SvdTrainTestExecutor,
        args={},
        model_name="svd"
    ),
    evaluator.TrainTestExecutorConfig(
        factory=trainers.KnnTrainTestExecutor,
        args={},
        model_name="knn"
    )
]

np.random.seed(SEED)
_evaluator = evaluator.Evaluator(_resp_fn_config, n_proc=4)
_res = _evaluator.evaluate(
    _evaluators, 
    a_sample_rate=3,
    test_size=0.1, 
    sample_sizes=[0.1]
)

Subprocess started.
Subprocess started.
Subprocess started.
Load data finished. Number of users: 138 Number of items: 769
Load data finished. Number of users: 138 Number of items: 769
Load data finished. Number of users: 138 Number of items: 769
IAutoRec.IAutoRec.IAutoRec.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Epoch: 0000; Epoch: 0000; Epoch: 0000; RMSE:0.7191212250166875; MAE:0.5704763710572504
RMSE:1.0349645666499936; MAE:0.8232820870737657RMSE:0.6317349215743885; MAE:0.48713349284780294

Epoch: 0003; Epoch: 0003; Epoch: 0003; RMSE:0.7383086277303282; MAE:0.5888399548544982
RMSE:1.0169446820947827; MAE:0.807503446857055
RMSE:0.6502273345719204; MAE:0.5431996288543961
Epoch: 0006; Epoch: 0006; Epoch: 0006; RMSE:0.7229238962107192; MAE:0.5774573034474599
RMSE:0.6297215928518438; MAE:0.5074209298666841RMSE:1.0209053502142658; MAE:0.8076131221236997

Epoch: 0009; Epoch: 0009; Epoch: 0009; RMSE:0.7200452681182796; MAE:0.5728463742360629
Epoch: 0012; RMSE:1.0198047374141688; MAE:0.8115425196472774RMSE:0.6194329451685662; MAE:0.48276681504727315

Epoch: 0012; Epoch: 0012; RMSE:0.7224515863799681; MAE:0.5737772808518883
Epoch: 0015; RMSE:0.6174692401202949; MAE:0.4843420695569124RMSE:1.014659102053104; MAE:0.8075056527094251

Epoch: 0015; Epoch: 0015; RMSE:0.7197259327857891; MAE:0.572667947646229

In [ ]:
_res

In [ ]:
error_surface = utils.group_points_by_minimum_error(_res)
error_surface

In [ ]:
from modules import utils
importlib.reload(utils)

import plotly.express as px

for ss in [0.1]:
    fig = px.scatter_3d(
        error_surface[error_surface["sample_size"] == ss], 
        x='a1', 
        y='a2', 
        z='rmse',
        size="rmse",
        size_max=18, 
        opacity=1,
        color="model_name",
        color_continuous_scale=px.colors.sequential.thermal[::-1]
    )

    fig.update_layout(
        margin=dict(l=20, r=20, t=20, b=20),
    )

    fig.show("notebook")

In [ ]:
viz = error_surface.rename(columns={"a1": "α1", "a2": "α2", "rmse": "RMSE", "model_name": "Model"})


colormap = {
    "knn": "yellow",
    "svd": "blue",
    "autorec": "red"
}

viz["color"] = viz["Model"].apply(lambda x: colormap[x])

In [ ]:
viz

In [ ]:
from plotly import graph_objects as go

for sample_size in [0.1, 0.2, 0.4, 0.6, 0.8, 1.0]:
    _exp_data = viz[viz["sample_size"] == sample_size]
    fig = go.Figure(
        layout=go.Layout(
            height=600,
            width=800,
            font=dict(size=16),
            title=dict(
                text=f"Model RMSE depending on α1 and α2 values. \n{sample_size}"
            ),
            margin=dict(l=20, r=20, t=20, b=20),
            scene=dict(
                xaxis = dict(title="α1"),
                yaxis = dict(title="α2"),
                zaxis = dict(title="RMSE")
            ),
            scene_camera=dict(
                up=dict(x=0, y=0, z=1),
                center=dict(x=0, y=0, z=0),
                eye=dict(x=1.2, y=1.8, z=1.0)
            )
        ),
        data=[
            go.Scatter3d(
                name=model_name,
                x=_exp_data[_exp_data["Model"] == model_name]["α1"],
                y=_exp_data[_exp_data["Model"] == model_name]["α2"],
                z=_exp_data[_exp_data["Model"] == model_name]["RMSE"],
                mode="markers",
                marker=dict(
                    size=6,
                    color=colormap[model_name],
                    line=dict(width=1, color='DarkSlateGrey')
                )
            ) for model_name in ["autorec", "svd", "knn"]
        ]
    )

    fig.show("notebook")

In [ ]:
    go.Scatter3d(
        x=model_data["a1"],
        y=model_data["a2"],
        z=model_data["rmse"], 
        mode="markers",
        marker=dict(
            size=4,
            color=colormap[model],
            line=dict(width=2, color='DarkSlateGrey')),
        name="thing"
    )